## Google colab setup

In [ ]:
try:
  import google.colab
  IS_COLAB = True
except:
  IS_COLAB = False

In [ ]:
if IS_COLAB:
  !pip install tensorflow_addons
  !pip install unidecode
  !pip install transformers
  !pip install nlpaug

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/SemiSupervised/*.csv .

## Imports and setup

In [ ]:
%%capture
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
import re
import time
import spacy
import numpy as np
from transformers import AutoTokenizer, TFAutoModel
from unidecode import unidecode

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')

## Utils

In [ ]:
data = pd.read_csv("train_ner.csv")

In [ ]:
def preprocess(x):
    s = unidecode(x)
    s = str.lower(s)
    s = re.sub(r"\[[a-z]+\]","", s)
    s = re.sub(r"\*","", s)
    s = re.sub(r"[^a-zA-Z0-9]+"," ",s)
    s = re.sub(r" +"," ",s)
    s = re.sub(r"(.)\1+",r"\1",s)

    return s

In [ ]:
label_ids = {label_name:i for i, label_name in enumerate(sorted(set(data["label"])))}
label_ids

{'ABUSE': 0, 'INSULT': 1, 'OTHER': 2, 'PROFANITY': 3}

In [ ]:
data["preprocessed"] = data["text"].apply(preprocess)
data.head()

,id,text,label,preprocessed
0,32674,da de unde stii u mai [ORG] ca banii au fost p...,ABUSE,da de unde sti u mai ca bani au fost pt si nu ...
1,16514,m*uie [PERS] m*uie [PERS] ... m*uie\nbai kakat...,INSULT,muie muie muie bai kakatule stai in banca ta d...
2,32556,PT ALA CARE ARE TREABA CU [PERS]!!ESTI UNUL CA...,OTHER,pt ala care are treaba cu esti unul care nu ar...
3,23861,sunt bucuros ca [PERS] nu a mai venit la [ORG]...,OTHER,sunt bucuros ca nu a mai venit la jucator de d...
4,21811,[PERS] esti....PE..N\n ES..CU..LI..BI.L!!! te ...,INSULT,esti pe n es cu li bi l te asemeni cu de la c...


In [ ]:
data["class"] = data["label"].map(lambda x: label_ids[x])
data.head(5)

,id,text,label,preprocessed,class
0,32674,da de unde stii u mai [ORG] ca banii au fost p...,ABUSE,da de unde sti u mai ca bani au fost pt si nu ...,0
1,16514,m*uie [PERS] m*uie [PERS] ... m*uie\nbai kakat...,INSULT,muie muie muie bai kakatule stai in banca ta d...,1
2,32556,PT ALA CARE ARE TREABA CU [PERS]!!ESTI UNUL CA...,OTHER,pt ala care are treaba cu esti unul care nu ar...,2
3,23861,sunt bucuros ca [PERS] nu a mai venit la [ORG]...,OTHER,sunt bucuros ca nu a mai venit la jucator de d...,2
4,21811,[PERS] esti....PE..N\n ES..CU..LI..BI.L!!! te ...,INSULT,esti pe n es cu li bi l te asemeni cu de la c...,1


In [ ]:
data["label"].value_counts()

OTHER        3649
ABUSE        2768
INSULT       2242
PROFANITY    1294
Name: label, dtype: int64

## Prepare data splits

In [ ]:
tok_robert = AutoTokenizer.from_pretrained("readerbench/RoBERT-base")
robert = TFAutoModel.from_pretrained("readerbench/RoBERT-base")

In [ ]:
def split_ssl_data(ids_array,mask_array,labels,num_classes,label_percent):
  labeled = None
  unlabeled = None

  for class_idx in range(num_classes):
    class_ids = ids_array[labels==class_idx]
    class_mask = mask_array[labels==class_idx]
    sz = int(label_percent * class_ids.shape[0])

    labels_reduced = labels[labels==class_idx][:sz]
    labeled_ids, unlabeled_ids = class_ids[:sz], class_ids[sz:]
    labeled_mask, unlabeled_mask = class_mask[:sz], class_mask[sz:]

    if not labeled:
      labeled = (labeled_ids, labeled_mask, labels_reduced)
      unlabeled = (unlabeled_ids, unlabeled_mask)
    else:
      labeled = (
          np.concatenate([labeled[0],labeled_ids]),
          np.concatenate([labeled[1],labeled_mask]),
          np.concatenate([labeled[2],labels_reduced])
      )
      unlabeled = (
          np.concatenate([unlabeled[0],unlabeled_ids]),
          np.concatenate([unlabeled[1],unlabeled_mask]),
      )

  return labeled, unlabeled


In [ ]:
def preprocess_robert(x):
  t = tok_robert(x,padding="max_length",max_length=96,truncation=True,return_tensors='np')
  return t["input_ids"], t["attention_mask"]

def map_func(input_ids, masks, labels):
  return {'input_ids': input_ids, 'attention_mask': masks}, labels

def map_func2(input_ids, masks):
  return {'input_ids': input_ids, 'attention_mask': masks}

def prepare_ds(filename,batch_size=64,label_percent=0.8):
  df = pd.read_csv(filename)
  X_id_mask = df['text'].map(preprocess).apply(preprocess_robert).apply(pd.Series)

  X_id_mask.columns = ["input_ids","attention_mask"]

  ids_array = np.squeeze(np.stack(X_id_mask.input_ids.values), axis=1)
  mask_array = np.squeeze(np.stack(X_id_mask.attention_mask.values), axis=1)
  labels = df["label"].map(lambda x: label_ids[x]).values

  res_ds = tf.data.Dataset.from_tensor_slices((ids_array, mask_array, labels)).map(map_func).shuffle(len(df)).batch(batch_size)
  return res_ds

In [ ]:
def prepare_train_ds(filename,batch_size=16,label_percent=0.05):
  df = pd.read_csv(filename)
  X_id_mask = df['text'].map(preprocess).apply(preprocess_robert).apply(pd.Series)

  X_id_mask.columns = ["input_ids","attention_mask"]

  ids_array = np.squeeze(np.stack(X_id_mask.input_ids.values), axis=1)
  mask_array = np.squeeze(np.stack(X_id_mask.attention_mask.values), axis=1)
  labels = df["label"].map(lambda x: label_ids[x]).values

  labeled, unlabeled = split_ssl_data(ids_array,mask_array,labels,len(label_ids),label_percent)
  labeled_ds = tf.data.Dataset.from_tensor_slices(labeled)
  labeled_ds = labeled_ds.map(map_func).shuffle(len(labeled_ds)).batch(batch_size).repeat()
  unlabeled_ds = tf.data.Dataset.from_tensor_slices(unlabeled)
  unlabeled_ds = unlabeled_ds.map(map_func2).shuffle(len(unlabeled_ds)).batch(4*batch_size).repeat()
  
  return labeled_ds, unlabeled_ds

In [ ]:
labeled_ds, unlabeled_ds = prepare_train_ds("train_ner.csv")
test_ds = prepare_ds("test_ner.csv")
val_ds = prepare_ds("validation_internal_ner.csv")

## Model definition and declaration

In [ ]:
class FreeMatch(tf.keras.Model):
  def __init__(self,bert_model,num_classes=4,**kwargs):
    super(FreeMatch,self).__init__(**kwargs)
    self.bert = bert_model
    self.bert.trainable = False
    self.num_classes = num_classes
    self.weak_augment = tf.keras.layers.GaussianNoise(stddev=0.5)
    self.strong_augment = tf.keras.layers.GaussianNoise(stddev=5)


    self.cls_head = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(
            tf.keras.layers.LSTM(10),
            merge_mode="ave"
        ),
        tf.keras.layers.Dense(units=self.num_classes,activation="softmax")
    ])


  def call(self, inputs, training):
    ids, mask = inputs
    
    embeds = self.bert(input_ids=ids, attention_mask=mask).last_hidden_state

    strongs = self.strong_augment(embeds,training=training)
    weaks = self.weak_augment(embeds,training=training)

    strong_preds = self.cls_head(strongs)
    weak_preds = self.cls_head(weaks)

    return weak_preds, strong_preds



In [ ]:
model = FreeMatch(bert_model=robert)

## Train and evaluate

In [ ]:
optim = tfa.optimizers.AdamW(weight_decay=0.001,learning_rate=0.005)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_prec_metric = tf.keras.metrics.Precision(name="precision")
val_recall_metric = tf.keras.metrics.Recall(name="recall")
f1_metric_micro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='micro', name='f1_micro')
f1_metric_macro = tfa.metrics.F1Score(num_classes=4, threshold=0.5, average='macro', name='f1_macro')

In [ ]:
thresh = 0.9
unsup_weight = 1.0
num_classes = len(label_ids)
fairness_weight = 0.01
eps = 1e-4
steps_per_epoch = 150

In [ ]:
tau = tf.Variable(tf.constant(1/num_classes))
p_thresh = tf.Variable(tau * tf.ones((num_classes,),dtype=tf.float32))
hist_t = tf.Variable(eps * tf.ones((num_classes,),dtype=tf.float32))
eps_t = tf.Variable(eps * tf.ones((num_classes,),dtype=tf.float32))

ema_decay = tf.constant(0.999)

In [ ]:
@tf.function
def sumnorm(x):
  return x/tf.reduce_sum(x)

@tf.function
def maxnorm(x):
  return x/tf.reduce_max(x)


@tf.function
def train_step(x,y,xu,ema_decay):
  with tf.GradientTape() as tape:
      wl,_ = model([x["input_ids"],x["attention_mask"]],training=True)
      wu,su = model([xu["input_ids"],xu["attention_mask"]],training=True)

# 2 from paper
      ls = loss_fn(y, wl)

# 3 
      local_thresh_update = tf.reduce_mean(tf.reduce_max(wu,axis=1))
      tau.assign(ema_decay * tau + (1-ema_decay) * local_thresh_update)

# 4
      p_thresh.assign(ema_decay * p_thresh + (1-ema_decay) * tf.reduce_mean(wu,axis=0))

# 5
      hist_t.assign(ema_decay * hist_t + (1-ema_decay) * tf.reduce_sum(tf.one_hot(tf.argmax(wu,axis=1),4),axis=0))

# 6
      sat = tau * maxnorm(p_thresh)

# 7
      mask = tf.reduce_max(wu,axis=1)>=tf.gather(sat,tf.argmax(wu,axis=1))
      
      su = su[mask]
      wu = wu[mask]
      

      lu = loss_fn(tf.argmax(wu,axis=1),su)
      
      p_avg = tf.reduce_mean(su,axis=0)
      h_avg = tf.reduce_sum(tf.one_hot(tf.argmax(su,axis=1),4),axis=0)

      lf = -tf.keras.metrics.categorical_crossentropy(sumnorm(p_thresh/hist_t),sumnorm(p_avg/(eps_t+h_avg)))

      loss = ls + unsup_weight * lu + fairness_weight * lf

  grads = tape.gradient(loss, model.trainable_weights)
  optim.apply_gradients(zip(grads, model.trainable_weights))

  return loss

@tf.function
def test_step(x, y):
    wpred,_ = model([x["input_ids"],x["attention_mask"]], training=False)
    val_acc_metric.update_state(y, wpred)
    true_hot = tf.one_hot(y, 4)
    val_prec_metric.update_state(true_hot, wpred)
    val_recall_metric.update_state(true_hot, wpred)
    f1_metric_micro.update_state(true_hot, wpred)
    f1_metric_macro.update_state(true_hot, wpred)

In [ ]:
EPOCHS=25
max_val_acc = 0.0
best_weights = None

l_iter = iter(labeled_ds)
u_iter = iter(unlabeled_ds)
for epoch in range(EPOCHS):
  print(f"Epoch {epoch}")
  start_time = time.time()

  for step in range(151):
    x, y = next(l_iter)
    xu = next(u_iter)
    
    loss = train_step(x,y,xu,ema_decay)
    if step % 50 == 0:
      print(
        "Training loss (for one batch) at step %d: %.4f"
        % (step, float(loss))
      )
  for x_batch_val, y_batch_val in val_ds:
      test_step(x_batch_val, y_batch_val)

  acc = float(val_acc_metric.result())
  prec = float(val_prec_metric.result())
  recall = float(val_recall_metric.result())
  micro = float(f1_metric_micro.result())
  macro = float(f1_metric_macro.result())

  val_acc_metric.reset_states()
  val_prec_metric.reset_states()
  val_recall_metric.reset_states()
  f1_metric_micro.reset_states()
  f1_metric_macro.reset_states()

  if acc > max_val_acc:
    max_val_acc = acc
    best_weights = model.get_weights()
  print(f"Validation acc: {acc:.3f} precision: {prec:.3f} recall: {recall:.3f} f1_micro: {micro:.3f} f1_macro: {macro:.3f}")
  print("Time taken: %.2fs" % (time.time() - start_time))
print("Restoring best weights relative to validation accuracy...")
model.set_weights(best_weights)

Epoch 0


2023-02-26 08:34:42.535591: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100


Training loss (for one batch) at step 0: 2.6781
Training loss (for one batch) at step 50: 2.0876
Training loss (for one batch) at step 100: 2.3032
Training loss (for one batch) at step 150: 2.2053
Validation acc: 0.510 precision: 0.654 recall: 0.236 f1_micro: 0.346 f1_macro: 0.163
Time taken: 76.35s
Epoch 1
Training loss (for one batch) at step 0: 1.7199
Training loss (for one batch) at step 50: 1.8361
Training loss (for one batch) at step 100: 2.3641
Training loss (for one batch) at step 150: 2.2281
Validation acc: 0.513 precision: 0.635 recall: 0.274 f1_micro: 0.383 f1_macro: 0.209
Time taken: 52.86s
Epoch 2
Training loss (for one batch) at step 0: 2.3985
Training loss (for one batch) at step 50: 2.3266
Training loss (for one batch) at step 100: 2.1774
Training loss (for one batch) at step 150: 2.0808
Validation acc: 0.505 precision: 0.569 recall: 0.396 f1_micro: 0.467 f1_macro: 0.295
Time taken: 52.78s
Epoch 3
Training loss (for one batch) at step 0: 2.1718
Training loss (for one ba

In [ ]:
for x_batch_val, y_batch_val in test_ds:
    test_step(x_batch_val, y_batch_val)
acc = float(val_acc_metric.result())
prec = float(val_prec_metric.result())
recall = float(val_recall_metric.result())
micro = float(f1_metric_micro.result())
macro = float(f1_metric_macro.result())

val_acc_metric.reset_states()
val_prec_metric.reset_states()
val_recall_metric.reset_states()
f1_metric_micro.reset_states()
f1_metric_macro.reset_states()
print(f"Test acc: {acc:.3f} precision: {prec:.3f} recall: {recall:.3f} f1_micro: {micro:.3f} f1_macro: {macro:.3f}")

Test acc: 0.539 precision: 0.593 recall: 0.484 f1_micro: 0.533 f1_macro: 0.385
